# O-RAW integration with XNAT collection

Using Zhenwei Shi's original O-RAW extension, build an integration with XNAT to read the contents of an XNAT Project and process them using pyradiomics, with the feature extraction settings given in a yaml file.

### import necessary components

In [2]:
###############################
#@author: zhenwei.shi, Maastro#
###############################
#
#ORAW modifications and integrations with XNAT by Leonard Wee July 2021

from __future__ import print_function

from time import process_time
import os,yaml
import ORAW
import glob
import shutil
import pandas as pd
import xnat
from DicomDatabase import DicomDatabase
from datetime import date

### define an upload file operation

In [3]:
def upload_file(session, project, subject, experiment, assessment, resource, csvfile):
    xnat_project = session.projects[project]
    xnat_subject = session.classes.SubjectData(parent=xnat_project, label=subject)
    xnat_experiment = session.classes.CtSessionData(parent=xnat_subject, label=experiment)
    xnat_resource = session.classes.ResourceCatalog(parent=xnat_experiment, label=resource)
    #xnat_assessment = session.classes.QcAssessmentData(parent=xnat_experiment, label=assessment) #not used
    #xnat_resource = session.classes.ResourceCatalog(parent=xnat_assessment, label=resource) #resource under experiment instead
    xnat_resource.upload(csvfile, os.path.basename(csvfile)) # upload
#    for file_ in data:
#        resource.upload(file_, os.path.basename(file_))
#    pass

### configure user-dependent settings

In [7]:
#------------------------- USER SETTINGS -------------------------------

# set up XNAT login credentials here
# .......................
xnatUrl = 'http://172.19.0.5:80'  #change me! e.g. 'http://localhost:8081/'
#xnatUrl = 'https://xnat.bmia.nl'
xnatUser = 'admin'  #change me!
#xnatUser = 'leonardwee'
xnatPass = 'admin'  #change me!
xnatProject = 'oraw-plastimatch-test'  #change me!

#----------------O-RAW initial parameters -------------------
roi = '[Gg][Tt][Vv]' #change me - to process every structure say 'all'
export_format = 'csv'
# export_format = 'rdf' #warn : not yet tested by Leonard
export_name = 'oraw'
#walk_dir = './demo_data/CT' #not used in XNAT mode

### retrieve list of patients and dicom experiments from XNAT project

In [8]:
with xnat.connect(xnatUrl, user=xnatUser, password=xnatPass) as session:
    myProject= session.projects[xnatProject]
    mySubjectsList = myProject.subjects.values()
    for s in mySubjectsList:
        mySubjectID = s.label
        mySubject = myProject.subjects[mySubjectID]
        myExperimentsList = mySubject.experiments.values()
        for e in myExperimentsList:
            myExperimentID = e.label
            myExperiment = mySubject.experiments[myExperimentID]
            print(xnatProject + "\t" + mySubjectID + "\t" + myExperimentID)

[WARNING] Found an unsupported version (1.8.2), trying 1.7 compatible model builder


oraw-plastimatch-test	LUNG1-002	LUNG1-002


### main section

In [ ]:
#-----------------create tmp CT/STRUCT directories-----------
CTWorkingDir = "./CTFolder"
STRUCTWorkingDir = "./StructFolder"
XNATdownload = "./XnatDownload"

if os.path.exists(CTWorkingDir):
  shutil.rmtree(CTWorkingDir)
if os.path.exists(STRUCTWorkingDir):
  shutil.rmtree(STRUCTWorkingDir)
if os.path.exists(XNATdownload):
  shutil.rmtree(XNATdownload)

if not os.path.exists(CTWorkingDir):
  os.makedirs(CTWorkingDir)
if not os.path.exists(STRUCTWorkingDir):
  os.makedirs(STRUCTWorkingDir)
if not os.path.exists(XNATdownload):
  os.makedirs(XNATdownload)

start_time = process_time()

#-----------------exclusion ROIs-----------
excludeStructRegex = "(Patient.*|BODY.*|Body.*|NS.*|Couch.*)"
if os.environ.get("EXCLUDE_STRUCTURE_REGEX") is not None:
    excludeStructRegex = os.environ.get("EXCLUDE_STRUCTURE_REGEX")

# ---------------output formats----------------------
if export_format == 'rdf':
    exportDir = './RFstore/Turtle_output' # export format is RDF
else:
    exportDir = './RFstore/CSV_output' # export format is CSV

# -----------------------------------------------------------
with xnat.connect(xnatUrl, user=xnatUser, password=xnatPass) as session:
    myProject= session.projects[xnatProject]
    mySubjectsList = myProject.subjects.values()
    for s in mySubjectsList:
        mySubjectID = s.label
        mySubject = myProject.subjects[mySubjectID]
        myExperimentsList = mySubject.experiments.values()
        for e in myExperimentsList:
            myExperimentID = e.label
            myExperiment = mySubject.experiments[myExperimentID]
            myExperiment.download_dir(XNATdownload)
            
            # initialize dicom DB
            dicomDb = DicomDatabase()
            dicomDb.parseFolder(XNATdownload)
        
            # main
            for ptid in dicomDb.getPatientIds():
                print("Processing: %s" % (ptid)) # get patient by ID
                myPatient = dicomDb.getPatient(ptid)
                # loop over RTStructs of this patient
                for myStructUID in myPatient.getRTStructs():
                    print("Starting with RTStruct %s" % myStructUID)
                    # Get RTSTRUCT by SOP Instance UID
                    myStruct = myPatient.getRTStruct(myStructUID)
                    # Get CT which is referenced by this RTStruct, and is linked to the same patient
                    # mind that this can be None, as only a struct, without corresponding CT scan is found
                    myCT = myPatient.getCTForRTStruct(myStruct)
                
                    # clear the working CT/STRUCT folder
                    if not (os.listdir(CTWorkingDir)==[] and os.listdir(STRUCTWorkingDir)==[]):
                        ct_files = glob.glob(os.path.join(CTWorkingDir,'*'))
                        for f in ct_files:
                            os.remove(f)
                        struct_files = glob.glob(os.path.join(STRUCTWorkingDir,'*'))
                        for f in struct_files:
                            os.remove(f)
                        
                    #only if we have both RTStruct and CT
                    if myCT is not None:
                        shutil.move(myStruct.getFileLocation(),os.path.join(STRUCTWorkingDir,'struct.dcm')) # move RTSTRUCT file to tmp folder as 'struct.dcm'
                        slices = myCT.getSlices()
                        for i in range(len(slices)):
                            shutil.move(slices[i],os.path.join(CTWorkingDir,str(i)+".dcm"))
                        if roi == 'all':
                            ORAW.executeORAWbatch_all([ptid],roi,myStructUID,exportDir,export_format,export_name,[CTWorkingDir],[STRUCTWorkingDir],excludeStructRegex)
                        else:
                            ORAW.executeORAWbatch_roi([ptid],roi,myStructUID,exportDir,export_format,export_name,[CTWorkingDir],[STRUCTWorkingDir],excludeStructRegex)
                    
                    #upload the generated object back into XNAT
                    resource_folder = export_name + '_' + date.today().strftime("%Y%m%d")
                    if export_format == 'rdf':
                        myFile = exportDir + '/' + ptid + '.ttl' # export format is RDF
                    else:
                        myFile = exportDir + '/' + ptid + '.csv' # export format is CSV
                    try:
                        upload_file(session, xnatProject, mySubjectID, myExperimentID, '0', resource_folder, myFile)
                        print("O-RAW output save as resource in XNAT under experiment.")
                        os.remove(f)
                    except:
                        print("O-RAW output failed to upload to XNAT!")
                    
                    #conclude and move onto next subject
                    print("Done for RTStruct %s of subject %s" % (myStructUID, ptid))
##### ----------------------------------------------------------------------------

stop_time = process_time()

#cleanup
shutil.rmtree(CTWorkingDir)
shutil.rmtree(STRUCTWorkingDir)
shutil.rmtree(XNATdownload)
